In [8]:
import ee
import geemap
import os
from tqdm import tqdm
import calendar
import json
from google.cloud import storage
import time

default_s2_bands = ['B2', 'B3', 'B4', 'B8',  # 10m
                    'B5', 'B8A', 'B11']  # 20m
bands10 = ['B2','B3','B4','B8']
bands20 = ['B5','B8A','B11']

PROJECT_NAME = 'treeai-470815'
EXP_SCALE = 10
MAX_PIXELS = 1e13

# Initialize the Earth Engine module and register this code in project 'TreeAI'
ee.Authenticate()
ee.Initialize(project=PROJECT_NAME)

In [9]:
def maskBySCL(s2_image, masked_type=[0, 1, 3, 7, 8, 9, 10, 11]):
    scl = s2_image.select('SCL')
    img_bilinear = s2_image.resample('bilinear')
    img_bilinear = img_bilinear.addBands(scl, overwrite=True)

    bad_values = ee.List(masked_type)
    bad_flag = scl.remap(bad_values, ee.List.repeat(1, bad_values.length()), 0)

    good_mask = bad_flag.neq(1)

    masked_img = img_bilinear.updateMask(good_mask)
    return masked_img

In [ ]:
swiss_boundary = ee.FeatureCollection("projects/treeai-470815/assets/swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET")
out_crs = swiss_boundary.geometry().crs()
year = 2018
month = 7

In [11]:
start_date = ee.Date.fromYMD(year, month, 1)
end_date = start_date.advance(1, 'month')

swiss_S2_images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(swiss_boundary.geometry()) \
                    .filterDate(start_date, end_date) \
                    .map(lambda im: maskBySCL(im))

comp10 = swiss_S2_images.select(bands10).median().clip(swiss_boundary)
comp20_native = swiss_S2_images.select(bands20).median().clip(swiss_boundary)

comp20_at10m = comp20_native.resample('bilinear').reproject(comp10.projection())
S2_10m = comp10.addBands(comp20_at10m).set({
    'year': year, 'month': month, 'strategy': 'SCL_median', 'scale': 10
})

In [15]:
ee.batch.Export.image.toCloudStorage(
    image=S2_10m,
    description=f"S2_CH_{year}_{month:02d}_median_SCL_10m_EPSG2056",
    bucket='your-bucket-name',
    fileNamePrefix=f"Sentinel-2/S2_CH_{year}_{month:02d}_median_SCL_10m_EPSG2056",
    region=swiss_boundary.geometry(),
    scale=10,
    crs='EPSG:2056',
    maxPixels=1e13,
    fileFormat='GeoTIFF',
    formatOptions={'cloudOptimized': True}
).start()